# AI Agent Profile Chatbot

This notebook creates an AI chatbot using OpenAI's GPT model to answer questions about a professional profile. It uses a system prompt with the user's background and LinkedIn data and is deployed with Gradio.

## Imports & config

In [4]:
from dotenv import load_dotenv
import openai
from pypdf import PdfReader
import gradio as gr
import os 

c:\Users\matte\source\AI-Agents-Camp\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_BASE_URL = os.getenv('OPENAI_BASE_URL', 'https://api.openai.com/v1')
OPENAI_MODEL = os.getenv('OPENAI_MODEL', 'gpt-4o-mini')

HEADERS = {
    'Authorization': f'Bearer {OPENAI_API_KEY}' if OPENAI_API_KEY else '',
    'Content-Type': 'application/json',
}

## Prompts and Resources loading
Load from disk a CV in pdf format and a presentation letter as text file.

In [6]:
reader = PdfReader("../assets/me/CV.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("../assets/me/about.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Matteo Rigoni"

In [7]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

system_prompt

"You are acting as Matteo Rigoni. You are answering questions on Matteo Rigoni's website, particularly questions related to Matteo Rigoni's career, background, skills and experience. Your responsibility is to represent Matteo Rigoni for interactions on the website as faithfully as possible. You are given a summary of Matteo Rigoni's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nSono uno sviluppatore full-stack, focalizzato sul backend in .NET e Azure. Lavoro su progetti enterprise — per lo più cloud, ma non solo — con un approccio pragmatico: soluzioni solide, senza complessità superflue.\n\nConsidero lo studio un requisito chiave di questo lavoro: l’apprendimento continuo si traduce in strumenti e pratiche che arricchiscono il mio bagaglio professionale.\n\nL’AI è ormai un alleato fisso nel mio 

In [8]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Test with gradio

In [35]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


## Evaluation of Agent's output

Guardrail on answer provided by LLM, if not acceptable, run again.

Suggested use of a different kind of LLM (as Gemini)

In [1]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [9]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [10]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
def evaluate(reply, message, history) -> Evaluation:
    resp = openai.responses.parse(
        model="gpt-4o-mini",
        input=[
            {"role": "system", "content": evaluator_system_prompt},
            {"role": "user", "content": evaluator_user_prompt(reply, message, history)},
        ],
        text_format=Evaluation,  
    )
    return resp.output_parsed

In [30]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [34]:
def chat_with_evaluation(message, history):    
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [35]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
